In [ ]:
static double diode_D_double[2];
static double diode_PM0_u[2] _ALIGN;
static double diode_PM0_prevU[2] _ALIGN;
static double diode_PM0_y[2] _ALIGN;
static size_t diode_PM0_topoIdx;
static char diode_PM0_withDiracs;
PM0_D_0_0 = [-1.0]

function PM0_natPreComm_0() return 1 end
function PM0_natPostComm_0() 
    u = diode_PM0_u;
    return (u[1] + u[2] < 0) 2 : 1
end

function PM0_forcedComm_0() return 1 end

PM0_D_0_1 = [-0.333333333333333204 -0.333333333333333204
             -0.999999999999999667  3.33333333333333261e-16]

function PM0_natPreComm_1() return 2 end
function PM0_natPostComm_1()
   const double * const u = diode_PM0_u;
   return (u[0] + u[1] >= 0.0) ? 1 : 2
end

function PM0_forcedComm_1() return 2 end

diode_PM0_topologies = [1, 2]

function diode_PM0_natComm()
    global diode_PM0_topoIdx
    diode_PM0_topoIdx = PM0_natPreComm[diode_PM0_topoIdx]()
    diode_PM0_topoIdx = PM0_natPostComm[diode_PM0_topoIdx]()
end

function diode_PM0_forcedComm()
    global diode_PM0_topoIdx
    diode_PM0_topoIdx = PM0_forcedComm[diode_PM0_topoIdx]();
end

function diode_PM0_output_0()
{
   const double * _RESTRICT D_0_data =
      _ASSUME_ALIGNED(diode_PM0_D_0_data[diode_PM0_topoIdx]);
   const size_t meterIdx[]={
      0,1
   };
   double y[2] _ALIGN;
   size_t i;
   for (i = 0; i < 2; ++i)
   {
      y[i] = 0;
      {
         const size_t *rowPtr = PM0_D_0_rowPtr[diode_PM0_topoIdx];
         const size_t *colIdx = PM0_D_0_colIdx[diode_PM0_topoIdx];
         size_t j;
         for (j = rowPtr[i]; j < rowPtr[i+1]; ++j)
            *(y+i) += D_0_data[j]*diode_PM0_u[colIdx[j]];
      }
   }
   for (i = 0; i < 2; ++i)
   {
      diode_PM0_y[meterIdx[i]] = y[i];
   }
}
                
diode_BlockOutputs diode_B;
diode_Parameters diode_P = {
   /* Parameter 'Amplitude' of
    *  Sine Wave Generator : 'diode/Sine Wave'
    */
   5.,
   /* Parameter 'Bias' of
    *  Sine Wave Generator : 'diode/Sine Wave'
    */
   0.
};
diode_ModelStates diode_X _ALIGN;
const double diode_sampleTime = 1e-11;
void diode_initialize(double time)
{
   /* Initialization for Subsystem : 'diode' */
   diode_X.diode_PM0_s = 0;
   diode_PM0_topoIdx = 0;
}

void diode_step(void)
{

   /* Electrical model */


   /* Electrical model input */
   /* Voltage Source (Controlled) : 'diode/V' */
   diode_PM0_u[0]=diode_B.SineWave;
   /* Diode : 'diode/D1' */
   diode_PM0_u[1]=0.0259999999999999988;
   /* End of electrical model input */


   /* Commutation */
   diode_PM0_natComm();
   diode_PM0_forcedComm();

   /* Electrical model output */
   diode_PM0_output_0();
   /* End of electrical model output */

   /* End of electrical model */
   if (diode_errorStatus)
   {
      return;
   }

   /* Update for Sine Wave Generator : 'diode/Sine Wave' */
   {
      double scaling, scaledX, scaledY;
      scaling = 1. +
                (0.5 *
       (diode_D_double[0]*diode_D_double[0] + diode_D_double[1]*
       diode_D_double[1] - 1.));
      scaledX = diode_D_double[0] / scaling;
      scaledY = diode_D_double[1] / scaling;
      diode_D_double[0] = 0.999921044203816112 * scaledX +
                          0.0125660398833526057 * scaledY;
      diode_D_double[1] = -0.0125660398833526057 * scaledX +
                          0.999921044203816112 * scaledY;
   }

   /* Update for Subsystem : 'diode' */
   memcpy(diode_PM0_prevU,diode_PM0_u,2*sizeof(double));
}

In [ ]:
const typ = Dual{Float64}

N = 2
M = 1

D = zeros(typ, N)
x = zeros(typ, N)
u = zeros(typ, M)
y = zeros(typ, M)
A = zeros(typ, N, N)
B0 = zeros(typ, N, M)
B1 = zeros(typ, N, M)
prevX = zeros(typ, N)
prevU = zeros(typ, M)

A =  [ 0.999950001249979126 0.0;
       0.0 0.998001998667333079]
B0 = [-0.000499983333645829132;
       0.000998667666133551055]
B1 = [-0.000499991666770832209;
       0.000999333666533382126]

function update!(x, A, B0, B1)
    global prevX
    global prevU
    global u
    
    x .=  A * prevX #=
    =#.+ B0.* prevU #=
    =#.+ B1.* u
    
    return nothing
end

firstRun = true
sampleTime = 0.01e-9
function initialize(time)
    global x
    global firstRun
    
    x .= [0.0, 0.0]
    firstRun = true

    return nothing
end


function step!()
    global firstRun
    global A
    global x
    global y
    global u
    global B0
    global B1
    global prevX
    
    if firstRun == true
        firstRun = false
    else
        prevX .= x
        update!(x, A, B0, B1)
    end
    
    y[1] = 1.0 * x[1] + 0.2 * x[2] - 0.2 * u[1]
    prevU .= u
end